# 01 Pytorch 基础环境配置

在学习 PyTorch 基础的安装和环境配置时，以下是一些关键的学习重点和步骤。这些内容将帮助您在 Mac M1 上成功设置和使用 PyTorch。

## PyTorch 基础安装和环境配置学习重点

### 1. **选择合适的 Python 发行版**

- **Miniforge**: 对于 Mac M1 用户，推荐使用 Miniforge，因为它支持 ARM 架构并且与 Conda 兼容。Miniforge 是一个轻量级的 Conda 发行版，适合在 Apple Silicon 上进行科学计算。

### 2. **创建 Conda 环境**

- **创建环境**:
  - 使用 Conda 创建一个新的 Python 环境，以便隔离项目依赖。
  - 命令示例：
    ```bash
    conda create -n pytorch_env python=3.10 -y
    ```
- **激活环境**:
  - 激活新创建的环境以确保后续安装的软件包仅影响该环境。
  - 命令示例：
    ```bash
    conda activate pytorch_env
    ```

### 3. **安装 PyTorch**

- **选择正确的版本**:
  - 确保安装支持 MPS 的 PyTorch 版本（例如 1.12 或更高）。
- **安装命令**:
  - 使用 Conda 安装 PyTorch 及其依赖库。
  - 命令示例：
    ```bash
    conda install pytorch torchvision torchaudio -c pytorch-nightly
    ```

### 4. **验证安装**

- **检查 PyTorch 版本**:

  - 在 Python 交互环境中检查 PyTorch 版本以确认安装成功。
  - 示例代码：
    ```python
    import torch
    print(torch.__version__)
    ```

- **验证 MPS 可用性**:
  - 确认 MPS 后端是否可用，以便在 Mac M1 上利用 GPU 加速。
  - 示例代码：
    ```python
    print(torch.backends.mps.is_available())
    print(torch.backends.mps.is_built())
    ```

### 5. **基本配置文件管理**

- **保存环境配置**:

  - 将 Conda 环境导出为 YAML 文件，方便在其他机器上重建相同环境。
  - 命令示例：
    ```bash
    conda env export > environment.yml
    ```

- **加载环境配置**:
  - 在其他机器上使用 YAML 文件创建相同的 Conda 环境。
  - 命令示例：
    ```bash
    conda env create -f environment.yml
    ```

### 学习重点总结

- 理解 Python 发行版选择的重要性，特别是在不同硬件架构上的兼容性。
- 掌握 Conda 环境管理，包括创建、激活、导出和导入。
- 熟悉 PyTorch 的基本安装步骤，确保 MPS 后端的可用性以利用 Mac M1 的硬件优势。
- 学会验证安装和配置，通过简单的 Python 脚本检查软件包和硬件支持。

通过掌握这些基础知识，您将能够为后续的深度学习项目打下坚实的基础，并有效利用 Mac M1 设备进行高效计算。


In [3]:
!python --version
!conda list

Python 3.10.14
# packages in environment at /opt/homebrew/Caskroom/miniforge/base/envs/ai_dev:
#
# Name                    Version                   Build  Channel
appnope                   0.1.4              pyhd8ed1ab_0    conda-forge
asttokens                 2.4.1              pyhd8ed1ab_0    conda-forge
blas                      1.0                    openblas    defaults
brotli-python             1.0.9           py310h313beb8_8    defaults
bzip2                     1.0.8                h80987f9_6    defaults
ca-certificates           2024.8.30            hf0a4a13_0    conda-forge
certifi                   2024.8.30       py310hca03da5_0    defaults
charset-normalizer        3.3.2              pyhd3eb1b0_0    defaults
comm                      0.2.2              pyhd8ed1ab_0    conda-forge
debugpy                   1.6.7           py310h313beb8_0    defaults
decorator                 5.1.1              pyhd8ed1ab_0    conda-forge
exceptiongroup            1.2.2              pyhd8e

In [4]:
import torch
print(torch.__version__)
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

2.6.0.dev20240927
True
True


## 简单的神经网络模型的脚本，验证环境是否正常

In [5]:

import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 检查MPS是否可用
if not torch.backends.mps.is_available():
    raise RuntimeError("MPS is not available. Please check your setup.")

# 设置设备为MPS
device = torch.device("mps")

# 定义简单的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


# 初始化模型并移动到MPS设备
model = SimpleNN().to(device)

# 定义损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)

# 准备数据集和数据加载器
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(
    root='mnist_data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 训练函数


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 将数据移动到MPS设备
        X, y = X.to(device), y.to(device)

        # 计算预测误差
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss_val = loss.item()
            current = batch * len(X)
            print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")


# 执行一次训练过程
train(train_loader, model, loss_fn, optimizer)

loss: 2.296487  [    0/60000]
loss: 2.310711  [ 6400/60000]
loss: 2.295340  [12800/60000]
loss: 2.296603  [19200/60000]
loss: 2.284129  [25600/60000]
loss: 2.282688  [32000/60000]
loss: 2.280830  [38400/60000]
loss: 2.277373  [44800/60000]
loss: 2.260589  [51200/60000]
loss: 2.267814  [57600/60000]
